In [ ]:
"""======H2_gpu========="""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci
from qiskit_aer import AerSimulator

# ---------- Define GPU VQE Runner ----------
def run_vqe_gpu(problem, mapper):
    simulator = AerSimulator(method='statevector', device='GPU', precision='single')
    simulator.set_options(
        max_parallel_threads=0,
        blocking_enable=True,
        blocking_qubits=10
    )
    estimator = BackendEstimator(backend=simulator)
    optimizer = COBYLA(maxiter=200)

    initial_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=initial_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# ---------- PySCF Classical Methods ----------
def get_pyscf_mol(d, basis='sto3g'):
    mol = gto.Mole()
    mol.atom = f"H 0 0 0; H 0 0 {d}"
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    energy, _ = cisolver.kernel()
    return energy

# ---------- Qiskit Problem Generator ----------
def get_qiskit_problem(d, basis='sto3g'):
    driver = PySCFDriver(
        atom=f"H 0 0 0; H 0 0 {d}",
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=basis
    )
    return driver.run()

# ---------- Main Code ----------
dists = np.linspace(0.4, 2.5, 14)
vqe_energies, uhf_energies, fci_energies = [], [], []

for i, d in enumerate(dists):
    print(f"\n--- Distance: {d:.2f} Å ---")

    # VQE on GPU
    try:
        problem = get_qiskit_problem(d)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe_gpu(problem, mapper)
        print(f"VQE (GPU): {vqe_energy:.6f}")
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE Error: {e}")

    # UHF
    try:
        mol = get_pyscf_mol(d)
        uhf_energy = run_uhf_pyscf(mol)
        print(f"UHF       : {uhf_energy:.6f}")
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF Error: {e}")

    # FCI
    try:
        fci_energy = run_fci_pyscf(mol)
        print(f"FCI       : {fci_energy:.6f}")
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI Error: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)

# ---------- Plot Results ----------
plt.figure(figsize=(6,4.5))
plt.plot(dists, vqe_energies, label='VQE (GPU)', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF', color='red')
plt.plot(dists, fci_energies, '--', label='FCI', color='black')
plt.xlabel("H-H Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("H₂ Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ---------- Print Min Energies ----------
print("\n========= Minimum Energies =========")
print(f"Min VQE Energy: {np.nanmin(vqe_energies):.6f} Hartree")
print(f"Min UHF Energy: {np.nanmin(uhf_energies):.6f} Hartree")
print(f"Min FCI Energy: {np.nanmin(fci_energies):.6f} Hartree")


In [ ]:
"""======Ne_gpu========"""

In [ ]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA


# === Function to create problem for Neon ===
def get_qiskit_problem(atom_spec, charge, spin, basis):
    driver = PySCFDriver(atom=atom_spec,
                         unit=DistanceUnit.ANGSTROM,
                         charge=charge,
                         spin=spin,
                         basis=basis)
    return driver.run()


# === Function to run GPU-accelerated VQE ===
def run_vqe_gpu(problem, mapper):
    # Create GPU simulator
    gpu_backend = AerSimulator(method='statevector',
                               device='GPU',
                               precision='single')

    estimator = BackendEstimator(backend=gpu_backend)

    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals,
                             problem.num_particles,
                             mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals,
                   problem.num_particles,
                   mapper,
                   initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer,
              initial_point=[0] * ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    # Get interpreted total energy
    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
basis = 'sto3g'
charge = 0
spin = 0
atom_spec = "Ne 0.0 0.0 0.0"  # Neon atom at origin

problem = get_qiskit_problem(atom_spec, charge, spin, basis)
mapper = ParityMapper(num_particles=problem.num_particles)

print("\nRunning GPU-accelerated VQE for Neon (Ne)...")
vqe_energy = run_vqe_gpu(problem, mapper)

print(f"\n=== Neon Ground State Energy (VQE, GPU) ===")
print(f"Energy: {vqe_energy:.6f} Hartree")


In [ ]:
'''========H2O gpu========'''

In [ ]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA


# === Function to set up H2O problem ===
def get_qiskit_problem(atom_spec, charge, spin, basis):
    driver = PySCFDriver(atom=atom_spec,
                         unit=DistanceUnit.ANGSTROM,
                         charge=charge,
                         spin=spin,
                         basis=basis)
    return driver.run()


# === Function to run GPU VQE ===
def run_vqe_gpu(problem, mapper):
    # GPU-enabled simulator
    gpu_backend = AerSimulator(method='statevector',
                               device='GPU',
                               precision='single')

    estimator = BackendEstimator(backend=gpu_backend)

    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals,
                             problem.num_particles,
                             mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals,
                   problem.num_particles,
                   mapper,
                   initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer,
              initial_point=[0] * ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
basis = 'sto3g'
charge = 0
spin = 0

# Experimental geometry for H2O (bent, bond angle ~104.45°)
atom_spec = """
O  0.000000   0.000000   0.000000
H  0.000000  -0.757160   0.586260
H  0.000000   0.757160   0.586260
"""

problem = get_qiskit_problem(atom_spec, charge, spin, basis)
mapper = ParityMapper(num_particles=problem.num_particles)

print("\nRunning GPU-accelerated VQE for H2O...")
vqe_energy = run_vqe_gpu(problem, mapper)

print(f"\n=== H2O Ground State Energy (VQE, GPU) ===")
print(f"Energy: {vqe_energy:.6f} Hartree")


In [ ]:
"""=====CH4 gpu========"""

In [ ]:
import numpy as np

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# --- methane geometry (tetrahedral approx, in Angstrom) ---
methane_geom = """
C  0.000000   0.000000   0.000000
H  0.629118   0.629118   0.629118
H -0.629118  -0.629118   0.629118
H -0.629118   0.629118  -0.629118
H  0.629118  -0.629118  -0.629118
"""

# --- Helper: build Qiskit Nature problem (PySCF driver + optional active space) ---
def build_problem(basis="sto3g", use_active_space=True,
                  active_electrons=8, active_orbitals=8):
    driver = PySCFDriver(
        atom=methane_geom,
        unit=DistanceUnit.ANGSTROM,
        basis=basis,
        charge=0,
        spin=0
    )
    problem = driver.run()

    if use_active_space:
        transformer = ActiveSpaceTransformer(
            num_electrons=active_electrons,
            num_spatial_orbitals=active_orbitals
        )
        problem = transformer.transform(problem)

    return problem

# --- GPU VQE runner ---
def run_vqe_on_gpu(problem, maxiter=200):
    # Create GPU simulator (statevector)
    sim = AerSimulator(method="statevector", device="GPU", precision="single")
    # Optional: tune
    sim.set_options(max_parallel_threads=0, blocking_enable=True, blocking_qubits=10)

    # Backend estimator wrapping the GPU simulator
    estimator = BackendEstimator(backend=sim)

    # Mapper + initial state + ansatz
    mapper = ParityMapper(num_particles=problem.num_particles)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(
        num_spatial_orbitals=problem.num_spatial_orbitals,
        num_particles=problem.num_particles,
        qubit_mapper=mapper,
        initial_state=init_state
    )

    # Optimizer and VQE
    optimizer = COBYLA(maxiter=maxiter)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    # Run VQE (on GPU via BackendEstimator)
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    # Interpret result to get total energy (chemical energy)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result

# --- Main: build, run, print ---
if __name__ == "__main__":
    # Build problem with a conservative active space to fit a laptop GPU.
    # Adjust active_orbitals upward if you have more VRAM.
    try:
        print("Building problem (PySCF)...")
        problem = build_problem(basis="sto3g", use_active_space=True,
                                active_electrons=8, active_orbitals=8)

        print("Problem built:")
        print(f"  spatial orbitals: {problem.num_spatial_orbitals}")
        print(f"  electrons: {problem.num_particles}")
        print("Launching VQE on GPU... (this may take a while)")
        energy, result = run_vqe_on_gpu(problem, maxiter=200)

        print("\n=== Methane (CH4) Ground State Energy (VQE, GPU) ===")
        print(f"Energy: {energy:.8f} Hartree")
        # optional: print some metadata
        if hasattr(result, "optimal_point"):
            print("Optimizer converged:", getattr(result, "optimal_value", "N/A"))
    except Exception as e:
        print("Error during GPU VQE run:", e)
        print("Suggestions:")
        print(" - Reduce active_space (fewer orbitals)")
        print(" - Use 'use_active_space=False' to run smaller test (may be too large)")
        print(" - Run on CPU with Estimator() if GPU driver issues occur")


In [ ]:
"""=========N2 gpu========="""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, fci

# ----------------- User parameters -----------------
BASIS = "sto3g"
# Active space (CAS): tune these to fit GPU memory. CAS(6,6) is a good starting point.
ACTIVE_ELECTRONS = 6
ACTIVE_SPATIAL_ORBITALS = 6

# Bond distances for N-N (Angstrom)
DISTANCES = np.linspace(0.9, 2.6, 12)

# VQE optimizer settings
MAXITER = 200
COBYLA_TOL = 1e-3
# ---------------------------------------------------

def build_full_pyscf_mol(distance, basis=BASIS):
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def run_fci_pyscf(mol):
    # FCI on full AO space may be impossible for large basis; try and catch in main loop
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    e, _ = cisolver.kernel()
    return e

def build_qiskit_problem(distance, basis=BASIS,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS):
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    driver = PySCFDriver(atom=atom, basis=basis, unit=DistanceUnit.ANGSTROM,
                         charge=0, spin=0)
    problem = driver.run()
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def dense_exact_energy_from_qubit_op(qubit_op):
    mat = qubit_op.to_matrix()
    evals = np.linalg.eigvalsh(mat)
    return float(np.min(np.real(evals)))

def run_vqe_on_gpu(problem, mapper, maxiter=MAXITER):
    # Create GPU statevector simulator
    sim = AerSimulator(method="statevector", device="GPU", precision="single")
    # Tuning options (optional)
    sim.set_options(max_parallel_threads=0, blocking_enable=True, blocking_qubits=10)

    backend_estimator = BackendEstimator(backend=sim)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    optimizer = COBYLA(maxiter=maxiter, tol=COBYLA_TOL)
    vqe = VQE(backend_estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result, qubit_op

# ----------------- Sweep and run -----------------
vqe_energies = []
uhf_energies = []
exact_energies = []
successful_points = 0

for d in DISTANCES:
    print(f"\n--- N-N distance = {d:.3f} Å ---")
    # classical references
    try:
        mol_full = build_full_pyscf_mol(d)
        uhf_e = run_uhf_pyscf(mol_full)
        print(f"UHF (PySCF): {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan

    # FCI on full AO (may fail / be too big)
    try:
        fci_e = run_fci_pyscf(mol_full)
        print(f"FCI (full AO): {fci_e:.8f}")
    except Exception as e:
        print("FCI (full) skipped/failed (likely too big):", e)
        fci_e = np.nan

    uhf_energies.append(uhf_e)

    # Build reduced Qiskit problem (active space)
    try:
        problem = build_qiskit_problem(d)
        mapper = ParityMapper(num_particles=problem.num_particles)
        print(f"Reduced problem: electrons={problem.num_particles}, spatial_orbitals={problem.num_spatial_orbitals}")
    except Exception as e:
        print("Building Qiskit problem failed:", e)
        vqe_energies.append(np.nan)
        exact_energies.append(np.nan)
        continue

    # Exact (dense) in reduced active space
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        exact_e = dense_exact_energy_from_qubit_op(qubit_op)
        print(f"Exact (active space): {exact_e:.8f}")
    except Exception as e:
        print("Exact (active space) failed:", e)
        exact_e = np.nan
    exact_energies.append(exact_e)

    # Run VQE on GPU
    try:
        vqe_e, vqe_result, _ = run_vqe_on_gpu(problem, mapper)
        print(f"VQE (GPU): {vqe_e:.8f}")
        successful_points += 1
    except Exception as e:
        print("VQE (GPU) failed:", e)
        vqe_e = np.nan

    vqe_energies.append(vqe_e)

# ----------------- Plotting -----------------
plt.figure(figsize=(7,5))
plt.plot(DISTANCES, vqe_energies, 'o-', label='VQE (GPU, active)', color='limegreen')
plt.plot(DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(DISTANCES, exact_energies, '--', label='Exact (active)', color='black')
plt.xlabel("N–N Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"N$_2$ Potential Energy Curve (active space: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ----------------- Report minima -----------------
if successful_points > 0 and not all(np.isnan(vqe_energies)):
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE (GPU) result from sweep ===")
    print(f"Distance = {DISTANCES[idx]:.3f} Å")
    print(f"VQE energy = {vqe_energies[idx]:.8f} Hartree")
else:
    print("\nNo successful VQE points to report.")


In [ ]:
'''==========O2 gpu ========='''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf

# ---------------- User parameters ----------------
BASIS = "sto3g"       # small basis for GPU speed
ACTIVE_ELECTRONS = 6  # reduce electrons for active space
ACTIVE_SPATIAL_ORBITALS = 6
DISTANCES = np.linspace(1.0, 2.6, 13)
MAXITER = 200
# -------------------------------------------------

def build_full_pyscf_mol(distance, basis=BASIS):
    atom = f"O 0 0 {-distance/2}; O 0 0 {distance/2}"
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0  # singlet; change to spin=2 for triplet O2
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def build_qiskit_problem(distance, basis=BASIS,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS):
    atom = f"O 0 0 {-distance/2}; O 0 0 {distance/2}"
    driver = PySCFDriver(atom=atom, basis=basis, unit=DistanceUnit.ANGSTROM,
                         charge=0, spin=0)
    problem = driver.run()
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    return transformer.transform(problem)

def dense_exact_energy_from_qubit_op(qubit_op):
    mat = qubit_op.to_matrix()
    evals = np.linalg.eigvalsh(mat)
    return float(np.min(np.real(evals)))

def run_vqe_gpu(problem, mapper, maxiter=MAXITER):
    # GPU backend
    backend = AerSimulator(device='GPU')
    estimator = Estimator(backend)
    optimizer = COBYLA(maxiter=maxiter)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper,
                   initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result

# --------- Sweep over bond distances ----------
vqe_energies = []
uhf_energies = []
exact_energies = []

for d in DISTANCES:
    print(f"\n--- O-O distance = {d:.3f} Å ---")

    # Classical UHF (full molecule)
    try:
        mol_full = build_full_pyscf_mol(d)
        uhf_e = run_uhf_pyscf(mol_full)
        print(f"UHF (PySCF) : {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # Build reduced problem
    try:
        problem = build_qiskit_problem(d)
        mapper = ParityMapper(num_particles=problem.num_particles)
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_energies.append(np.nan)
        continue

    # Exact (dense) in reduced space
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        exact_e = dense_exact_energy_from_qubit_op(qubit_op)
        print(f"Exact (active space) : {exact_e:.8f}")
    except Exception as e:
        print("Exact (active space) failed:", e)
        exact_e = np.nan
    exact_energies.append(exact_e)

    # GPU-accelerated VQE
    try:
        vqe_e, _ = run_vqe_gpu(problem, mapper, maxiter=MAXITER)
        print(f"VQE (GPU)            : {vqe_e:.8f}")
    except Exception as e:
        print("VQE failed:", e)
        vqe_e = np.nan
    vqe_energies.append(vqe_e)

# ---------- Plot ----------
plt.figure(figsize=(7,5))
plt.plot(DISTANCES, vqe_energies, 'o-', label='VQE (active, GPU)', color='limegreen')
plt.plot(DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(DISTANCES, exact_energies, '--', label='Exact (active)', color='black')
plt.xlabel("O–O Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"O$_2$ Potential Energy Curve (active: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o, GPU)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ---------- Minimum VQE value ----------
if np.all(np.isnan(vqe_energies)):
    print("\nNo successful VQE energies calculated.")
else:
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE result (GPU) ===")
    print(f"Distance = {DISTANCES[idx]:.3f} Å")
    print(f"VQE energy = {vqe_energies[idx]:.8f} Hartree")


In [ ]:
'''==========Ethyne (C₂H₂) gpu ========='''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# PySCF classical references (optional)
from pyscf import gto, scf, fci

# ----------------- User parameters -----------------
BASIS = "sto3g"
# Active space: tune to your GPU memory. Try smaller values (4,4) if you hit OOM.
ACTIVE_ELECTRONS = 8
ACTIVE_SPATIAL_ORBITALS = 8

# Sweep C–C distances (Å)
CC_DISTANCES = np.linspace(1.0, 1.6, 7)

# VQE optimizer settings
MAXITER = 150
COBYLA_TOL = 1e-3
# ---------------------------------------------------

def build_full_pyscf_mol(cc_distance, ch_distance=1.06, basis=BASIS):
    """
    Linear ethyne along z-axis:
    C1 at -cc_distance/2, C2 at +cc_distance/2
    H atoms bonded to each C outward along axis.
    """
    c1_z = -cc_distance / 2
    c2_z = cc_distance / 2
    h1_z = c1_z - ch_distance
    h2_z = c2_z + ch_distance

    atom = (
        f"C 0 0 {c1_z}; "
        f"C 0 0 {c2_z}; "
        f"H 0 0 {h1_z}; "
        f"H 0 0 {h2_z}"
    )
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def build_qiskit_problem(cc_distance, basis=BASIS,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS,
                        ch_distance=1.06):
    atom = (
        f"C 0 0 {-cc_distance/2}; "
        f"C 0 0 {cc_distance/2}; "
        f"H 0 0 {-cc_distance/2 - ch_distance}; "
        f"H 0 0 {cc_distance/2 + ch_distance}"
    )
    driver = PySCFDriver(atom=atom,
                         unit=DistanceUnit.ANGSTROM,
                         basis=basis,
                         charge=0,
                         spin=0)
    problem = driver.run()

    # Apply active-space reduction (CAS-like)
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def run_vqe_gpu(problem, mapper, maxiter=MAXITER):
    # GPU statevector simulator
    sim = AerSimulator(method="statevector", device="GPU", precision="single")
    # Optional tuning — helps memory/threading behavior
    sim.set_options(max_parallel_threads=0, blocking_enable=True, blocking_qubits=10)

    backend_estimator = BackendEstimator(backend=sim)

    # initial state and ansatz
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    optimizer = COBYLA(maxiter=maxiter, tol=COBYLA_TOL)
    vqe = VQE(backend_estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result, qubit_op

# ----------------- Sweep and run -----------------
vqe_energies = []
uhf_energies = []
exact_active_energies = []

for cc in CC_DISTANCES:
    print(f"\n=== C–C distance: {cc:.3f} Å ===")

    # Classical UHF (full molecule) — optional, quick reference
    try:
        mol_full = build_full_pyscf_mol(cc)
        uhf_e = scf.UHF(mol_full).kernel()
        print(f"UHF (PySCF): {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # Build reduced Qiskit problem
    try:
        problem = build_qiskit_problem(cc)
        mapper = ParityMapper(num_particles=problem.num_particles)
        print(f"Reduced problem: electrons={problem.num_particles}, spatial_orbitals={problem.num_spatial_orbitals}")
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_active_energies.append(np.nan)
        continue

    # Exact diagonalization in the reduced active space (dense)
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        mat = qubit_op.to_matrix()
        eigs = np.linalg.eigvalsh(mat)
        exact_e = float(np.min(np.real(eigs)))
        print(f"Exact (active space): {exact_e:.8f}")
    except Exception as e:
        print("Exact (active space) failed:", e)
        exact_e = np.nan
    exact_active_energies.append(exact_e)

    # Run VQE on GPU
    try:
        vqe_e, vqe_result, _ = run_vqe_gpu(problem, mapper, maxiter=MAXITER)
        print(f"VQE (GPU) energy   : {vqe_e:.8f}")
    except Exception as e:
        print("VQE (GPU) failed:", e)
        vqe_e = np.nan

    vqe_energies.append(vqe_e)

# ----------------- Plot results -----------------
plt.figure(figsize=(7,5))
plt.plot(CC_DISTANCES, vqe_energies, 'o-', label='VQE (GPU, active)', color='limegreen')
plt.plot(CC_DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(CC_DISTANCES, exact_active_energies, '--', label='Exact (active)', color='black')
plt.xlabel("C–C Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"C$_2$H$_2$ (Ethyne) Potential Energy Curve (active: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ----------------- Print minimum energy -----------------
if not all(np.isnan(vqe_energies)):
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE (GPU) result from sweep ===")
    print(f"C–C distance = {CC_DISTANCES[idx]:.3f} Å")
    print(f"VQE (GPU) energy = {vqe_energies[idx]:.8f} Hartree")
else:
    print("No successful VQE points to report.")


In [ ]:
"""=======Ethene (C₂H₄) GPU========="""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# PySCF classical references (optional)
from pyscf import gto, scf

# ----------------- User parameters -----------------
BASIS = "sto3g"
# Active space: tune to your GPU memory. Try smaller values (4,4) if you hit OOM.
ACTIVE_ELECTRONS = 8
ACTIVE_SPATIAL_ORBITALS = 8

# Sweep C–C distances (Å)
CC_DISTANCES = np.linspace(1.2, 1.6, 7)

# VQE optimizer settings
MAXITER = 150
COBYLA_TOL = 1e-3
# ---------------------------------------------------

def build_full_pyscf_mol(cc_distance, ch_distance=1.09, basis=BASIS):
    """
    Planar ethene geometry:
    C1 at -cc/2, C2 at +cc/2, two H atoms attached to each C in-plane.
    """
    c1_z = -cc_distance / 2
    c2_z = cc_distance / 2
    # put H at +/- along y for planarity (simple approx)
    h_y = ch_distance
    atom = (
        f"C 0 0 {c1_z}; "
        f"C 0 0 {c2_z}; "
        f"H 0 {h_y} {c1_z}; "
        f"H 0 {-h_y} {c1_z}; "
        f"H 0 {h_y} {c2_z}; "
        f"H 0 {-h_y} {c2_z}"
    )
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def build_qiskit_problem(cc_distance, basis=BASIS,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS,
                        ch_distance=1.09):
    atom = (
        f"C 0 0 {-cc_distance/2}; "
        f"C 0 0 {cc_distance/2}; "
        f"H 0 {ch_distance} {-cc_distance/2}; "
        f"H 0 {-ch_distance} {-cc_distance/2}; "
        f"H 0 {ch_distance} {cc_distance/2}; "
        f"H 0 {-ch_distance} {cc_distance/2}"
    )
    driver = PySCFDriver(atom=atom,
                         unit=DistanceUnit.ANGSTROM,
                         basis=basis,
                         charge=0,
                         spin=0)
    problem = driver.run()

    # Apply active-space reduction (CAS-like)
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def run_vqe_gpu(problem, mapper, maxiter=MAXITER):
    # Initialize GPU statevector simulator
    try:
        sim = AerSimulator(method="statevector", device="GPU", precision="single")
        # Optional tuning; helps memory/thread behavior
        sim.set_options(max_parallel_threads=0, blocking_enable=True, blocking_qubits=10)
    except Exception as e:
        raise RuntimeError(f"Failed to initialize GPU AerSimulator: {e}")

    backend_estimator = BackendEstimator(backend=sim)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    optimizer = COBYLA(maxiter=maxiter, tol=COBYLA_TOL)
    vqe = VQE(backend_estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result

# ----------------- Sweep and run -----------------
vqe_energies = []
uhf_energies = []
exact_active_energies = []

for cc in CC_DISTANCES:
    print(f"\n=== C–C distance: {cc:.3f} Å ===")

    # Classical UHF (full molecule) — optional quick reference
    try:
        mol_full = build_full_pyscf_mol(cc)
        uhf_mf = scf.UHF(mol_full)
        uhf_e = uhf_mf.kernel()
        print(f"UHF (PySCF): {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # Build reduced Qiskit problem
    try:
        problem = build_qiskit_problem(cc)
        mapper = ParityMapper(num_particles=problem.num_particles)
        print(f"Reduced problem: electrons={problem.num_particles}, spatial_orbitals={problem.num_spatial_orbitals}")
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_active_energies.append(np.nan)
        continue

    # Exact diagonalization in reduced active space (dense)
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        mat = qubit_op.to_matrix()
        eigs = np.linalg.eigvalsh(mat)
        exact_e = float(np.min(np.real(eigs)))
        print(f"Exact (active space): {exact_e:.8f}")
    except Exception as e:
        print("Exact (active space) failed:", e)
        exact_e = np.nan
    exact_active_energies.append(exact_e)

    # Run VQE on GPU
    try:
        vqe_e, vqe_result = run_vqe_gpu(problem, mapper, maxiter=MAXITER)
        print(f"VQE (GPU) energy   : {vqe_e:.8f}")
    except Exception as e:
        print("VQE (GPU) failed:", e)
        vqe_e = np.nan

    vqe_energies.append(vqe_e)

# ----------------- Plot results -----------------
plt.figure(figsize=(7,5))
plt.plot(CC_DISTANCES, vqe_energies, 'o-', label='VQE (GPU, active)', color='limegreen')
plt.plot(CC_DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(CC_DISTANCES, exact_active_energies, '--', label='Exact (active)', color='black')
plt.xlabel("C–C Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"C$_2$H$_4$ (Ethene) Potential Energy Curve (active: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ----------------- Print minimum energy -----------------
if not all(np.isnan(vqe_energies)):
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE (GPU) result from sweep ===")
    print(f"C–C distance = {CC_DISTANCES[idx]:.3f} Å")
    print(f"VQE (GPU) energy = {vqe_energies[idx]:.8f} Hartree")
else:
    print("No successful VQE points to report.")
